NLP Project for KU Course

In [16]:
import pandas as pd
import re
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
import nltk
from nltk.util import ngrams
nltk.download('punkt')
nltk.download('punkt_tab')


## Load the dataset
splits = {'train': 'train.parquet', 'validation': 'validation.parquet'}
df_train = pd.read_parquet("hf://datasets/coastalcph/tydi_xor_rc/" + splits["train"])
df_val = pd.read_parquet("hf://datasets/coastalcph/tydi_xor_rc/" + splits["validation"])


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/andreasmelbye/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/andreasmelbye/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


## Week 36

In [17]:
## Remove unwanted characters from the questions

def cleanDf(df):
    pattern = re.compile(r"[?؟,;\/\\\[\]#():]")
    df['question'] = df['question'].apply(lambda x: pattern.sub("", x))
    return df

In [18]:
langForStat = ['ar','ko','te']

numQuestions = []
totalWordCount = []
distinctWordCount = []
distinctCharCount = []

df_train_clean = cleanDf(df_train)
df_val_clean = cleanDf(df_val)

for lang in langForStat:
    numQuestions_train = df_train_clean[df_train_clean['lang'] == lang].shape[0]
    numQuestions_val = df_val_clean[df_val_clean['lang'] == lang].shape[0]
    numQuestions.append((lang, numQuestions_train, numQuestions_val))
    print(f"Language: {lang}, Train Questions: {numQuestions_train}, Validation Questions: {numQuestions_val}")

    # Compute word and character statistics
    df_train_lang = df_train_clean[df_train_clean['lang'] == lang].copy()
    df_val_lang = df_val_clean[df_val_clean['lang'] == lang]
    df_train_lang['wordcount'] = df_train_lang['question'].apply(lambda x: len(x.split()) if isinstance(x, str) else 0)


    maxId = df_train_lang['wordcount'].idxmax()
    longest_question = df_train_lang.loc[maxId, "question"]
    max_words = df_train_lang.loc[maxId, "wordcount"]

    # print(f"Language: {lang}")
    # print(f"  Longest train question (index {maxId}): {longest_question}")
    # print(f"  Word count: {max_words}")

    totalWordCount_train = df_train_lang['question'].apply(lambda x: len(x.split())).sum()
    totalWordCount_val = df_val_lang['question'].apply(lambda x: len(x.split())).sum()
    totalWordCount.append((lang, totalWordCount_train, totalWordCount_val))
    print(f"Language: {lang}, Train Total Words: {totalWordCount_train}, Validation Total Words: {totalWordCount_val}")



Language: ar, Train Questions: 2558, Validation Questions: 415
Language: ar, Train Total Words: 16202, Validation Total Words: 2621
Language: ko, Train Questions: 2422, Validation Questions: 356
Language: ko, Train Total Words: 11840, Validation Total Words: 1729
Language: te, Train Questions: 1355, Validation Questions: 384
Language: te, Train Total Words: 7668, Validation Total Words: 2299


In [19]:
def wordCount(df, lang):
    allWords = []
    df = df[df['lang'] == lang].copy()
    
    # df = df[df['answerable'] == False]
    df['question'] = df['question'].astype(str)

    for q in df['question']:
        allWords.extend(q.split()) 
    
    wordDict = dict(Counter(allWords))
    wordDict = dict(sorted(wordDict.items(), key=lambda item: item[1], reverse=True))

    return wordDict
            

for lang in langForStat:
    wordDict = wordCount(df_train_clean, lang)
    distinctWordCount.append((lang, len(wordDict)))
    print(f"Language: {lang}, Distinct Words: {len(wordDict)}")

    allChars = []
    for word in wordDict.keys():
        allChars.extend(list(word))
    
    charDict = dict(Counter(allChars))
    charDict = dict(sorted(charDict.items(), key=lambda item: item[1], reverse=True))
    distinctCharCount.append((lang, len(charDict)))
    print(f"Language: {lang}, Distinct Characters: {len(charDict)}")
    calculatedTotalWords = sum(wordDict.values())
    print(f"Language: {lang}, Calculated Total Words from Distinct Words: {calculatedTotalWords}")

    top5Words = list(wordDict.items())[:10]
    print(f"Language: {lang}, Top 5 Words: {top5Words}")

Language: ar, Distinct Words: 5427
Language: ar, Distinct Characters: 106
Language: ar, Calculated Total Words from Distinct Words: 16202
Language: ar, Top 5 Words: [('في', 592), ('من', 584), ('متى', 535), ('ما', 441), ('هو', 349), ('هل', 329), ('هي', 268), ('كم', 256), ('عدد', 161), ('أول', 157)]
Language: ko, Distinct Words: 4394
Language: ko, Distinct Characters: 819
Language: ko, Calculated Total Words from Distinct Words: 11840
Language: ko, Top 5 Words: [('가장', 527), ('무엇인가', 497), ('언제', 336), ('몇', 234), ('어디인가', 228), ('큰', 194), ('누구인가', 186), ('세상에서', 142), ('누구인가요', 105), ('무엇인가요', 95)]
Language: te, Distinct Words: 2420
Language: te, Distinct Characters: 91
Language: te, Calculated Total Words from Distinct Words: 7668
Language: te, Top 5 Words: [('ఎవరు', 274), ('ఏది', 192), ('ఎన్ని', 165), ('ఎప్పుడు', 154), ('ఏ', 142), ('ఎంత', 116), ('చిత్ర', 97), ('ఎక్కడ', 96), ('మొదటి', 86), ('ఉంది', 83)]


In [20]:
def arabicClassifier(question, context):
    goodWords = ['متى','ما','هو','هي','كم','عدد','أول','في']
    badWords = ['هل', 'يمكن']
    if any(word in question for word in badWords):
        return False
    if any(word in question for word in goodWords):
        return True
    else:
        return True
        # return np.random.choice([True, False])

def koreanClassifier(question, context):
    goodWords = ['가장', '무엇인가', '언제', '몇']
    badWords = ['수 '] # '시차는', '중력과'
    if any(word in question for word in badWords):
        return False
    if any(word in question for word in goodWords):
        return True
    else:
        return True
        # return np.random.choice([True, False])
    
def teluguClassifier(question, context):
    goodWords = []
    badWords = ['విస్తీర్ణం', 'జనాభా', 'ఆఫ్రికాలో']
    if any(word in question for word in badWords):
        return False
    if any(word in question for word in goodWords):
            return True
    # if any(word in question for word in badWords):
    #     return False
    else:
        return True
        # return np.random.choice([True, False])

### --- Arabic ---
arabicDf = df_val_clean[df_val_clean['lang'] == 'ar'].copy()
arabicDf['prediction'] = arabicDf.apply(lambda row: arabicClassifier(row['question'], row['context']), axis=1)
accuracy = (arabicDf['answerable'] == arabicDf['prediction']).mean()
print(f"Arabic Classifier Accuracy (validation): {accuracy * 100:.2f}%")
print(f"True distribution in validation set: {arabicDf['answerable'].value_counts(normalize=True).to_dict()}")

### --- Korean ---
koreanDf = df_val_clean[df_val_clean['lang'] == 'ko'].copy()
koreanDf['prediction'] = koreanDf.apply(lambda row: koreanClassifier(row['question'], row['context']), axis=1)
accuracy = (koreanDf['answerable'] == koreanDf['prediction']).mean()
print(f"Korean Classifier Accuracy (validation): {accuracy * 100:.2f}%")
print(f"True distribution in validation set: {koreanDf['answerable'].value_counts(normalize=True).to_dict()}")

### --- Telugu ---
teluguDf = df_val_clean[df_val_clean['lang'] == 'te'].copy()
teluguDf['prediction'] = teluguDf.apply(lambda row: teluguClassifier(row['question'], row['context']), axis=1)
accuracy = (teluguDf['answerable'] == teluguDf['prediction']).mean()
print(f"Telugu Classifier Accuracy (validation): {accuracy * 100:.2f}%")
print(f"True distribution in validation set: {teluguDf['answerable'].value_counts(normalize=True).to_dict()}")

Arabic Classifier Accuracy (validation): 96.87%
True distribution in validation set: {True: 0.8746987951807229, False: 0.12530120481927712}
Korean Classifier Accuracy (validation): 94.94%
True distribution in validation set: {True: 0.9466292134831461, False: 0.05337078651685393}
Telugu Classifier Accuracy (validation): 79.17%
True distribution in validation set: {True: 0.7578125, False: 0.2421875}


In [21]:
def overlap_ratio(question, context):
    q_words = set(question.split())
    c_words = set(context.split())
    if not q_words: return 0
    return len(q_words & c_words) / len(q_words)


import re
def contains_digit(text):
    return bool(re.search(r"\d+", text))


def arabicClassifier(question, context):
    goodWords = ['متى','ما','هو','هي','كم','عدد','أول','في']
    badWords = ['هل', 'يمكن']
    
    # Rule 1: keyword spotting
    if any(word in question for word in badWords):
        return False
    if any(word in question for word in goodWords):
        return True
    
    # # Rule 2: overlap
    # if overlap_ratio(question, context) < 0.1:
    #     return False
    
    # # Rule 3: digits
    # if contains_digit(question) and not contains_digit(context):
    #     return False
    
    # Rule 4: context length
    if len(context.split()) < 25:
        return False
    
    return True

# Arabic
arabicDf = df_val_clean[df_val_clean['lang'] == 'ar'].copy()
arabicDf['prediction'] = arabicDf.apply(lambda row: arabicClassifier(row['question'], row['context']), axis=1)
arabic_acc = (arabicDf['answerable'] == arabicDf['prediction']).mean()
print(f"Arabic Classifier Accuracy (validation): {arabic_acc*100:.2f}%")




Arabic Classifier Accuracy (validation): 96.87%


In [22]:
df_val_clean

,question,context,lang,answerable,answer_start,answer,answer_inlang
0,ఒరెగాన్ రాష్ట్రంలోని అతిపెద్ద నగరం ఏది,Portland is the largest city in the U.S. state...,te,True,0,Portland,None
1,కలరా వ్యాధిని మొదటగా ఏ దేశంలో కనుగొన్నారు,"The word cholera is from ""kholera"" from χολή ""...",te,True,99,Indian subcontinent,None
2,కలరా వ్యాధిని మొదటగా ఏ దేశంలో కనుగొన్నారు,Since it became widespread in the 19th century...,te,True,451,England,None
3,మొదటి ప్రపంచ యుద్ధం ఎప్పుడు మొదలయింది,World War I occurred from 1914 to 1918. In ter...,te,True,26,1914,None
4,మొదటి ప్రపంచ యుద్ధం ఎప్పుడు మొదలయింది,"World War I (often abbreviated as WWI or WW1),...",te,True,155,28 July 1914,None
...,...,...,...,...,...,...,...
3006,2011 జనగణన ప్రకారం రెయ్యలగడ్ద గ్రామములో పురుషు...,Reyyalagadda is a village belonging to Gangara...,te,True,378,37,37
3007,2011 జనాభా లెక్కల ప్రకారం బూతుమిల్లిపాడు గ్రామ...,Boothumillipadu is a village in Gannavaram man...,te,True,308,433,433
3008,2011 జనాభా లెక్కల ప్రకారం మల్లవేముల గ్రామ జనాభ...,Mallavemula is a village belonging to Chagalam...,te,False,-1,1131,1131
3009,2011 నాటికి రష్యా దేశ ప్రధాన మంత్రి ఎవరు,"Andria Urushadze (; born April 25, 1968) is a ...",te,False,-1,Vladimir Putin,వ్లాదిమిర్ పుతిన్


## Week 37

In [27]:
arabicDf_train = df_train_clean[df_train_clean['lang'] == 'ar'].copy()
teluguDf_train = df_train_clean[df_train_clean['lang'] == 'te'].copy()
koreanDf_train = df_train_clean[df_train_clean['lang'] == 'ko'].copy()

In [ ]:
from nltk import FreqDist, ConditionalFreqDist

# Make a list of all bigrams in arabicDf questions
allBigrams = []
for q in arabicDf_train['question']:
    tokens = nltk.word_tokenize(q)
    bigrams = list(ngrams(tokens, 2))
    allBigrams.extend(bigrams)
    
fdist = FreqDist(allBigrams)
print(fdist.most_common(10))
cfdist = ConditionalFreqDist((bigram[0], bigram) for bigram in allBigrams)
print(cfdist['ما'].most_common(10))
    


[(('ما', 'هي'), 232), (('من', 'هو'), 219), (('كم', 'عدد'), 114), (('ما', 'هو'), 113), (('متى', 'تم'), 103), (('متى', 'تأسست'), 54), (('في', 'أي'), 53), (('في', 'العالم'), 52), (('أي', 'عام'), 42), (('هل', 'يمكن'), 35)]
[(('ما', 'هي'), 232), (('ما', 'هو'), 113), (('ما', 'أول'), 9), (('ما', 'اسم'), 9), (('ما', 'هى'), 6), (('ما', 'نسبة'), 5), (('ما', 'أكبر'), 5), (('ما', 'عدد'), 5), (('ما', 'جنسية'), 5), (('ما', 'سبب'), 4)]
Top 10 Bigrams in Arabic Questions: [(('ما', 'هي'), 232), (('من', 'هو'), 219), (('كم', 'عدد'), 114), (('ما', 'هو'), 113), (('متى', 'تم'), 103), (('متى', 'تأسست'), 54), (('في', 'أي'), 53), (('في', 'العالم'), 52), (('أي', 'عام'), 42), (('هل', 'يمكن'), 35)]
[('متى', 'تدخلت'), ('تدخلت', 'روسيا'), ('روسيا', 'في'), ('في', 'الحرب'), ('الحرب', 'الأهلية'), ('الأهلية', 'السورية'), ('متى', 'حصلت'), ('حصلت', 'هنغاريا'), ('هنغاريا', 'على'), ('على', 'استقلالها'), ('استقلالها', 'من'), ('من', 'النمسا'), ('متى', 'تحالفت'), ('تحالفت', 'فرنسا'), ('فرنسا', 'و'), ('و', 'بريطانيا'), ('بريطا